In [89]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

In [90]:
!pip install apyori


In [91]:
df = pd.read_csv('u.data',sep='\t',names=['user_id','item_id','ratings','time'])

In [92]:
df.head()

,user_id,item_id,ratings,time
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [93]:
movie_titles = pd.read_csv("Movie_Id_Titles")

In [94]:
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [95]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

,user_id,item_id,ratings,time,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [96]:
df.groupby('user_id')['title'].count()

user_id
0        3
1      272
2       62
3       54
4       24
      ... 
939     49
940    107
941     22
942     79
943    168
Name: title, Length: 944, dtype: int64

In [97]:
user_watchlist=[]
for i in df['user_id']:
  user = df[df['user_id']==i]['title']
  user_watchlist.append(user.astype(str).values.tolist())


In [98]:
user_watchlist[0]

['Star Wars (1977)',
 'Empire Strikes Back, The (1980)',
 'Gone with the Wind (1939)']

In [99]:
from apyori import apriori
rules = apriori(transactions = user_watchlist, min_support = 0.1, min_confidence = 0.25, min_lift = 3, min_length = 2, max_length = 2)

In [100]:
rules

<generator object apriori at 0x7fea423ec830>

In [101]:
results = list(rules)

In [102]:
len(results)

79

In [103]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

In [104]:
resultsinDataFrame

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,Ace Ventura: Pet Detective (1994),Dumb & Dumber (1994),0.152305,0.585853,3.225981
1,Ace Ventura: Pet Detective (1994),Hot Shots! Part Deux (1993),0.159855,0.614893,3.235527
2,Ace Ventura: Pet Detective (1994),Robin Hood: Men in Tights (1993),0.111387,0.428456,3.033623
3,Ace Ventura: Pet Detective (1994),Tommy Boy (1995),0.136946,0.526771,3.051715
4,Addams Family Values (1993),Hot Shots! Part Deux (1993),0.133536,0.586396,3.085576
...,...,...,...,...,...
74,Star Trek V: The Final Frontier (1989),Star Trek: The Motion Picture (1979),0.127786,0.876475,3.352205
75,Three Colors: Blue (1993),Three Colors: Red (1994),0.128796,0.895066,5.001916
76,Three Colors: Blue (1993),Three Colors: White (1994),0.113197,0.786657,5.395987
77,Three Colors: Red (1994),Three Colors: White (1994),0.129596,0.724225,4.967737


In [105]:
resultsinDataFrame.nlargest(n = 10, columns = 'Lift')

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
76,Three Colors: Blue (1993),Three Colors: White (1994),0.113197,0.786657,5.395987
75,Three Colors: Blue (1993),Three Colors: Red (1994),0.128796,0.895066,5.001916
77,Three Colors: Red (1994),Three Colors: White (1994),0.129596,0.724225,4.967737
53,"Grand Day Out, A (1992)","Wrong Trousers, The (1993)",0.127136,0.945279,4.195771
32,"Close Shave, A (1995)","Grand Day Out, A (1992)",0.102617,0.548682,4.079543
18,Bananas (1971),Sleeper (1973),0.115837,0.788510,4.060839
65,It Happened One Night (1934),"Treasure of the Sierra Madre, The (1948)",0.107387,0.672659,3.867306
58,High Noon (1952),"Treasure of the Sierra Madre, The (1948)",0.120886,0.655479,3.768533
14,Apt Pupil (1998),As Good As It Gets (1997),0.147416,0.681806,3.758274
36,Demolition Man (1993),Hard Target (1993),0.105057,0.425861,3.639950
